In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
import pandas as pd
pd.set_option('max_colwidth',200)
raw_data = pd.read_pickle("/content/drive/MyDrive/project_pop/raw_data_df.pkl")

In [ ]:
raw_data.head()

,lyrics
Madonna,[Intro]\nLife is a mystery\nEveryone must stand alone\nI hear you call my name\nAnd it feels like home\n\n[Chorus]\nWhen you call my name\nIt's like a little prayer\nI'm down on my knees\nI want t...
Michael Jackson,"[Verse 1]\nShe was more like a beauty queen from a movie scene\nI said, ""Don't mind, but what do you mean, I am the one\nWho will dance on the floor in the round?""\nShe said I am the one\nWho will..."
Rihanna,"[Chorus: Rihanna]\nWork, work, work, work, work, work\nHe said me haffi\nWork, work, work, work, work, work\nHe see me do mi\nDirt, dirt, dirt, dirt, dirt, dirt\nSo me put in\nWork, work, work, wo..."
Taylor Swift,"[Verse 1]\nVintage tee, brand new phone\nHigh heels on cobblestones\nWhen you are young, they assume you know nothing\nSequin smile, black lipstick\nSensual politics\nWhen you are young, they assu..."
The Beatles,"[Verse 1]\nWhen I find myself in times of trouble, Mother Mary comes to me\nSpeaking words of wisdom, ""Let it be""\nAnd in my hour of darkness, she is standing right in front of me\nSpeaking words ..."


In [ ]:
# function to cleare dataframe
import re
import string

def clean_text_round1(text):
    '''Remove text in square brackets, remove commas, remove numbers and make text lowercase.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [ ]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(raw_data.lyrics.apply(round1))
data_clean.head()

,lyrics
Madonna,\nlife is a mystery\neveryone must stand alone\ni hear you call my name\nand it feels like home\n\n\nwhen you call my name\nits like a little prayer\nim down on my knees\ni want to take you there\...
Michael Jackson,\nshe was more like a beauty queen from a movie scene\ni said dont mind but what do you mean i am the one\nwho will dance on the floor in the round\nshe said i am the one\nwho will dance on the fl...
Rihanna,\nwork work work work work work\nhe said me haffi\nwork work work work work work\nhe see me do mi\ndirt dirt dirt dirt dirt dirt\nso me put in\nwork work work work work work\nwhen you ah guh\nlear...
Taylor Swift,\nvintage tee brand new phone\nhigh heels on cobblestones\nwhen you are young they assume you know nothing\nsequin smile black lipstick\nsensual politics\nwhen you are young they assume you know n...
The Beatles,\nwhen i find myself in times of trouble mother mary comes to me\nspeaking words of wisdom let it be\nand in my hour of darkness she is standing right in front of me\nspeaking words of wisdom let ...


In [ ]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', ' ', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [ ]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.lyrics.apply(round2))
data_clean.head()

,lyrics
Madonna,life is a mystery everyone must stand alone i hear you call my name and it feels like home when you call my name its like a little prayer im down on my knees i want to take you there in the mid...
Michael Jackson,she was more like a beauty queen from a movie scene i said dont mind but what do you mean i am the one who will dance on the floor in the round she said i am the one who will dance on the floor i...
Rihanna,work work work work work work he said me haffi work work work work work work he see me do mi dirt dirt dirt dirt dirt dirt so me put in work work work work work work when you ah guh learn learn l...
Taylor Swift,vintage tee brand new phone high heels on cobblestones when you are young they assume you know nothing sequin smile black lipstick sensual politics when you are young they assume you know nothing...
The Beatles,when i find myself in times of trouble mother mary comes to me speaking words of wisdom let it be and in my hour of darkness she is standing right in front of me speaking words of wisdom let it b...


In [ ]:
data_clean.to_pickle("data_clean_df.pkl")

<h4>Lemmetization</4>

In [ ]:
# import Lemmatizer
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Lemmatize with POS Tag
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
# my final list of pop artists whose album will be analysed
pops = ['Madonna', 'Michael Jackson', 'Rihanna', 'The Beatles', 'Taylor Swift']

# see the dataset size before applying lemmatizer
for p in pops:
    l1 =len(data_clean.lyrics[p])
    print(p + " len = " + str(l1))

Madonna len = 19822
Michael Jackson len = 22498
Rihanna len = 17798
The Beatles len = 11689
Taylor Swift len = 20656


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
# 1. Init Lemmatizer
lemmatizer = WordNetLemmatizer()

# 2. Lemmatize dataset
for pop, text in data_clean.iterrows():
    sentence = text[0]
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)])
    data_clean.lyrics[pop] = lemmatized_output
    lemmatized_output = ''

In [ ]:
# see the dataset size after applying lemmatizer
for p in pops:
    l1 =len(data_clean.lyrics[p])
    print(p + " len = " + str(l1))

Madonna len = 19218
Michael Jackson len = 21858
Rihanna len = 17246
The Beatles len = 11223
Taylor Swift len = 19892


<h4>Organizing Data</h4>

In [ ]:
trash = 'im cause like know dont aahah aaow ah ahah ahahah ahahahahahahahaha ahahahh	ahhahahah	ahhahaowww	amo ass aye ayy ayyuh bette cha chadachashashashashasha citys cocacola comin cyar cómo dijo eh eheh fikki gahhlee gimme givin gjoob ha haffi haoh haooh harlow hayworth hee heehee heeheehee heh hideandseek hmm ho hoo hov huh iana iiiiiiiiihiii ima imma inez juju kickin lalalalalalala la ma mckenzie meh mm mmm mmmm mmmmm mmmmmm mmmmmmmmmmmmmmm mojo molly naa nanana nananana nicest nonono nononononono nuh oath nononononono ohno ohoh ohohoh ol ono ooh oohooh oooh ooooooohhh ahoo just shit thats aint make fuck say na time man nigga right yeah life yourselves yous yukmouth yummy yumyum yyeah papapapapapapapa papapapapa owwwwww ow ouw shoochachoocha shoochachoo wah  yyyou zach zachs zakia zee zhané zion zsa aa aaah aaahhh aaliyah aames aanything ça écoute époque étaient était bneler nanamıyorum scratch şte beni hatırladın mı yedi infaz beni hatırladın mı hiç bir pişmanlığım yok beni hatırladın mı ben fazla güçlüyüm beni hatırladın mı bombaları atıyorum hiroşimaya atılanlar gibi   bu onun için x seni delirtirim çünkü ben mikrofonu kapıp ortadan kaybolabilirim down sendromu gibi gizlen ve insanların içinde dolaş sınırları olmayan biriyim hanginiz beni tanımlayabilir evrenin sağlığı için hiç bir prova olmadan bu tartışmalı kelimeleri sızdırıyorum yarışmak isteyeceğin tiplerden değilim çünkü çünkü senin kıçına çarpıyorum hani şu reklamdaki sürtüğe çarpan tren gibi bu reklam tvde yasaklandı ağır keskin nişancıyım seni vuruyorum başının ortadan ikiye ayrıldığını seyredebilirsin lokomotif gibi geliyoruz çılgın bir şekildeün devamı i̇şte içi boş  kalibrelik kurşunlar   beni hatırladın mı silahlarınızı havaya atın beni hatırladın mı çarp çarp beni hatırladın mı zenci geri çekil amına koyim beni hatırladın mı chkachkaonyx  zenciler sorularına hayır cevabını aldılar evet hayır dedim ama bu sanki hayırhayırhayır gibiydi hayat bir sürtük tanıştım onunla eğer zincirleri onun eline verirsen siktir git karşıma çıkmak istiyorsan daha iyinin daha iyisinin de daha iyisi olmalısın i̇şte ben ki usta rapçi akıttığı kanlara bakıyor sen ölünün de ölüsüsün artık sana para değil doktor gelecek sayemde kavgalı olduklarıma sırayla silahımla beraber gidiyorum kanıt mı hayır asla ardımda iz bırakmam ne kadar rapçi varsa hepsinin ruhuna az çok sahibim sev ya da nefret et ama durum bu annem müzik piyasası tarafından tecavüze uğradı ve ben doğdum ben en manyak zenciyim sana söylemiştim şu lezbiyen kaltaklardan oluşan total grubundan bile fazla kaltağa sahibim benle dalaşmak mı istiyorsun pbbt ölüm kontratını imzalamış olursun benim adım bu zencinin bunu söylediğine i̇nanamıyorum olmalıydı büyük ihtimalle katil olmadığımı söyleyecekler ama ben kendimi öldürüyorum ölüm içiyorum mutfak tezgahında topluluk ortasında kaltakları kondomsuz beceriyorum yani sana yaptığım şeyleri zerre umursamam çevremde bir tutam ot karşılığı tutabileceğim çok kiralık katil var ne yapmak istersin i̇bneler biz glock taşıyan adamlarız polis bizi kelepçeleyene kadar gürültü başlatırız ve uçak bombası bırakırız zaman doldu fahişeler bize dokunamazsınız beni öldürmek isteyenlerin kafamı isteyenlerin olduğunu duyuyorum bunların benim söyleyebileceğim şeyler olduğunu mu sanıyorsun   beni hatırladın mı sikimde değil beni hatırladın mı evet seni de sikeyim beni hatırladın mı kısa ve kurnazım beni hatırladın mı ben shadyim  ne zaman dışarı çıksam ateş etmeye devam ediyorum vatan için savaşmak anlamında değil kulübe her gittiğim anı kastettim gerizekalı sikik imajımı düzeltmeye çalışıyorum bu yüzden de sikik eleştirmenlere söz veriyorum ki altı dakika boyunca sikik demeyeceğim  dakika slim shady başla bebeğimin annesi beni kızgın bir sarışın yaptı bu sinir bana bir şarkı bile yaptırdı şarkıda onu öldürüyordum ve şarkıya hailieyi de koydum belki de hatalıydım hep böyle delice soruları düşünüp duruyorum beynimin içinde ama ben çılgın bi anneyle de sıkışıp kalmıştım gerçekten annen dediğin kadar hap kullanıyor mu eve geldim arkadaki camları birileri kırmış olmalı ve iki tane dolu makineli tüfeğimi ve de her iki yağmurluğumu çalmışlardı kötü kötü rüyalar görüyorum piknik sahnelerinden i̇ki velet var on altı yaşındalar  silahları ve onar tane dolu şarjörleri var ve altışar çocuk bunların yüzünden ölüyor bill clintonın sokakları güya temizleyeceğini söylediği konuşmalarında slim de suçlandı sikerim böyle işi pbbt – siz ibneler volkanda yanıp kül olabilirsiniz ve de cehennemde bir şişe benzin ve çakmakla yeniden ortaya çıkabilirsiniz aftermath dre silahı kap nerede olduğunu göster neye bakmıştın zenci   beni hatırlayamadın mı beni hatırladın mı beni hatırladın mı beni hatırladın mı guegueguess whos back hi mom \u200b\u200bscratch\u200b\u200b guess whos back \u200b\u200bscratch\u200b\u200b gue— \u200b\u200bscratch\u200b\u200b guess whos back  \u200b\u200bscratch\u200b\u200b guess whos back gue gueguegue guess whos back dr dre \u200b\u200bscratch\u200b\u200b guess whos back back back \u200b\u200bscratch\u200b\u200b back \u200b\u200bscratch\u200b\u200b'
trash_words = trash.split()

In [ ]:
# create document-term matrix for data_clean(lemmetized) dataframe
# also with stop words
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer

# Add new stop words
stop_words = text.ENGLISH_STOP_WORDS.union(trash_words)

# Recreate document-term matrix
cv = CountVectorizer(stop_words=trash_words)
data_cv = cv.fit_transform(data_clean.lyrics)
clean_data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
clean_data_dtm.index = data_clean.index

# pickle collected dataset for later use
clean_data_dtm.to_pickle("clean_data_df.pkl")

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
ls

clean_data_df.pkl  clean_data_dtm.pkl  data_clean_df.pkl  drive/  sample_data/


In [ ]:
clean_data_dtm

,abnormal,about,across,act,action,actress,add,adoration,adventure,advice,advise,afour,afraid,after,again,against,age,aggravation,ago,agree,air,ajoob,alarm,alarmed,albert,alien,alive,all,allan,allegation,alls,almost,alone,along,already,alright,altercation,always,amount,an,...,without,woah,woke,woman,won,wonder,wont,woo,woos,word,work,world,worried,worry,worth,would,wouldnt,wouldve,wow,write,wrong,yalls,year,yell,yellowmatter,yes,yesterday,yet,yo,you,youd,youll,young,your,youre,yours,yourself,youve,yuh,él
Madonna,1,2,0,1,0,0,0,0,0,1,0,2,2,1,0,1,0,0,0,0,2,0,0,0,0,0,0,25,0,0,0,0,2,0,2,0,0,3,0,4,...,1,0,0,0,0,0,1,0,0,3,0,37,0,0,0,5,0,0,0,0,1,0,0,0,0,1,2,0,0,141,1,1,3,76,12,0,4,3,0,1
Michael Jackson,0,19,3,0,0,0,0,0,0,1,0,0,0,1,6,1,0,1,0,0,1,0,0,0,0,1,2,21,0,3,0,1,2,0,0,3,0,4,0,1,...,1,0,0,0,0,1,4,1,0,2,0,12,1,1,0,2,2,0,0,0,9,1,1,0,0,0,0,0,0,215,0,2,2,43,15,1,10,5,0,0
Rihanna,0,3,0,2,1,0,0,1,0,0,1,0,0,0,0,2,0,0,1,0,1,0,0,1,0,0,5,20,0,0,0,0,0,1,0,0,1,7,0,3,...,5,12,0,0,0,0,0,3,0,1,79,2,0,0,0,3,0,0,0,0,3,0,0,1,0,0,0,0,6,147,0,1,0,15,16,0,0,3,1,0
Taylor Swift,0,10,0,0,0,4,1,0,1,0,0,0,0,1,2,0,1,0,0,0,1,0,0,0,0,0,0,45,0,0,1,0,3,0,0,1,0,5,2,3,...,1,0,0,1,0,0,1,0,0,3,0,2,0,0,3,13,0,9,0,4,1,0,1,0,0,1,0,1,0,252,13,3,9,71,17,4,0,1,0,0
The Beatles,0,2,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,3,1,0,1,0,0,31,1,0,0,0,0,1,0,0,0,0,0,8,...,0,0,1,0,1,0,0,0,1,8,1,3,0,0,0,1,2,0,2,1,2,0,3,0,1,0,9,0,0,41,0,2,0,12,3,0,0,2,0,0


In [ ]:
clean_data_dtm.to_pickle("clean_data_dtm.pkl")

In [ ]:
#a = clean_data_dtm
#type(a)

In [ ]:
#import pandas as pd
#a.to_excel('export_dataframe.xlsx')
